In [195]:
# Import packages
import pandas as pd
import numpy as np 
import requests 
import zipfile
import psycopg2
import sql_functions as sf
import sqlalchemy

### Import & examine crops & livestock production data

In [38]:
# Create a function for downloading the data

def data_download():
     
    r = requests.get(url)
    with open(path+zip_file, 'wb') as f:
        f.write(r.content)

    data = zipfile.ZipFile(path+zip_file, 'r')
    data.extract(csv_file, path)

In [39]:
# Define the parameters for the download

url = f'https://fenixservices.fao.org/faostat/static/bulkdownloads/Production_Crops_Livestock_E_All_Data_(Normalized).zip'
zip_file = 'Production_Crops_Livestock_E_All_Data_(Normalized).zip'
path = './data/'
csv_file = 'Production_Crops_Livestock_E_All_Data_(Normalized).csv'

In [41]:
# Run the data download function 
data_download()

In [42]:
# Import crops & livestock data for all countries
production = pd.read_csv(path+csv_file, encoding='latin-1', low_memory=False)

In [43]:
production.head()

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1975,1975,ha,0.0,E,NaN
1,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1976,1976,ha,5900.0,E,NaN
2,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1977,1977,ha,6000.0,E,NaN
3,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1978,1978,ha,6000.0,E,NaN
4,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1979,1979,ha,6000.0,E,NaN


In [17]:
production.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3761168 entries, 0 to 3761167
Data columns (total 14 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Area Code        int64  
 1   Area Code (M49)  object 
 2   Area             object 
 3   Item Code        int64  
 4   Item Code (CPC)  object 
 5   Item             object 
 6   Element Code     int64  
 7   Element          object 
 8   Year Code        int64  
 9   Year             int64  
 10  Unit             object 
 11  Value            float64
 12  Flag             object 
 13  Note             object 
dtypes: float64(1), int64(5), object(8)
memory usage: 401.7+ MB


In [18]:
# Check for full duplicates
production.duplicated().value_counts()

False    3761168
Name: count, dtype: int64

In [19]:
# Check for NAs
production.isnull().value_counts()

Area Code  Area Code (M49)  Area   Item Code  Item Code (CPC)  Item   Element Code  Element  Year Code  Year   Unit   Value  Flag   Note 
False      False            False  False      False            False  False         False    False      False  False  False  False  True     3673453
                                                                                                                                    False      87715
Name: count, dtype: int64

In [20]:
# List unique values in the Note column
print(production["Note"].unique())

# Drop the 'Note' column as it does not contain relevant info
production = production.drop(columns = 'Note')

[nan 'Unofficial figure']


'Year Code' and 'Year' columns contain the same data, so we can drop one of them. 

In [21]:
# Drop 'Year code' column
production = production.drop(columns = 'Year Code')

### Import emissions from crops data

In [44]:
url = f'https://fenixservices.fao.org/faostat/static/bulkdownloads/Emissions_crops_E_All_Data_(Normalized).zip'
zip_file = 'Emissions_crops_E_All_Data_(Normalized).zip'
csv_file = 'Emissions_crops_E_All_Data_(Normalized).csv'

In [45]:
# Run the data download function 
data_download()

In [46]:
# Import crops emissions data for all countries
emissions_crops = pd.read_csv(path+csv_file, encoding='latin-1', low_memory=False)

In [47]:
emissions_crops.head()

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Year Code,Year,Source Code,Source,Unit,Value,Flag,Note
0,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1961,1961,3050,FAO TIER 1,kt,0.1141,E,NaN
1,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1962,1962,3050,FAO TIER 1,kt,0.1141,E,NaN
2,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1963,1963,3050,FAO TIER 1,kt,0.1141,E,NaN
3,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1964,1964,3050,FAO TIER 1,kt,0.1145,E,NaN
4,2,'004,Afghanistan,44,'0115,Barley,72430,Crops total (Emissions N2O),1965,1965,3050,FAO TIER 1,kt,0.1145,E,NaN


### Import emissions from livestock data

In [48]:
url = f'https://fenixservices.fao.org/faostat/static/bulkdownloads/Emissions_livestock_E_All_Data_(Normalized).zip'
zip_file = 'Emissions_livestock_E_All_Data_(Normalized).zip'
csv_file = 'Emissions_livestock_E_All_Data_(Normalized).csv'

In [49]:
# Run the data download function 
data_download()

In [50]:
# Import crops emissions data for all countries
emissions_livestock = pd.read_csv(path+csv_file, encoding='latin-1', low_memory=False)

In [51]:
emissions_livestock.head()

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Year Code,Year,Source Code,Source,Unit,Value,Flag,Note
0,2,'004,Afghanistan,1107,'02132,Asses,5111,Stocks,1961,1961,3050,FAO TIER 1,An,1300000.0,A,NaN
1,2,'004,Afghanistan,1107,'02132,Asses,5111,Stocks,1962,1962,3050,FAO TIER 1,An,851850.0,A,NaN
2,2,'004,Afghanistan,1107,'02132,Asses,5111,Stocks,1963,1963,3050,FAO TIER 1,An,1001112.0,A,NaN
3,2,'004,Afghanistan,1107,'02132,Asses,5111,Stocks,1964,1964,3050,FAO TIER 1,An,1150000.0,E,NaN
4,2,'004,Afghanistan,1107,'02132,Asses,5111,Stocks,1965,1965,3050,FAO TIER 1,An,1300000.0,A,NaN


### Import food balances data

In [151]:
url = f'https://fenixservices.fao.org/faostat/static/bulkdownloads/FoodBalanceSheets_E_All_Data_(Normalized).zip'
zip_file = 'FoodBalanceSheets_E_All_Data_(Normalized).zip'
csv_file = 'FoodBalanceSheets_E_All_Data_(Normalized).csv'

In [53]:
# Run the data download function 
data_download()

In [152]:
# Import crops emissions data for all countries
food_balances = pd.read_csv(path+csv_file, encoding='latin-1', low_memory=False)

In [128]:
# Get table info
food_balances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4320908 entries, 0 to 4320907
Data columns (total 13 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Area Code        int64  
 1   Area Code (M49)  object 
 2   Area             object 
 3   Item Code        int64  
 4   Item Code (FBS)  object 
 5   Item             object 
 6   Element Code     int64  
 7   Element          object 
 8   Year Code        int64  
 9   Year             int64  
 10  Unit             object 
 11  Value            float64
 12  Flag             object 
dtypes: float64(1), int64(5), object(7)
memory usage: 428.6+ MB


In [153]:
# Since the info above does not provide anything apart from the data type, get additional info on unique and missing values
print("Unique values and missing values(%) of each column:")

food_balances_info= pd.DataFrame({"Unique values": food_balances.nunique(),
                            "Missing values(%)": round(food_balances.isnull().sum()/food_balances.shape[0]*100, 2)
                            }).rename_axis('Columns', axis='rows')                       

food_balances_info

Unique values and missing values(%) of each column:


,Unique values,Missing values(%)
Columns,,
Area Code,221,0.0
Area Code (M49),221,0.0
Area,221,0.0
Item Code,123,0.0
Item Code (FBS),123,0.0
Item,120,0.0
Element Code,21,0.0
Element,21,0.0
Year Code,12,0.0


In [80]:
# Check for full duplicates
food_balances.duplicated().value_counts()

False    4320908
Name: count, dtype: int64

In [158]:
food_balances.head()

,area_code,area,item_code,item,element_code,element,year,unit,value,flag
0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,2010,1000 No,28189.67,X
1,2,Afghanistan,2501,Population,511,Total Population - Both sexes,2011,1000 No,29249.16,X
2,3,Albania,2501,Population,511,Total Population - Both sexes,2010,1000 No,2913.40,X
3,3,Albania,2501,Population,511,Total Population - Both sexes,2011,1000 No,2900.65,X
4,2,Afghanistan,2501,Population,511,Total Population - Both sexes,2012,1000 No,30466.48,X


In [154]:
# Drop unneeded columns
food_balances.drop(columns = ['Year Code','Area Code (M49)', 'Item Code (FBS)'], inplace=True)

In [157]:
# Rename remaining columns
food_balances.rename(columns={'Area Code':'area_code', 'Area':'area', 'Item Code':'item_code', 'Item':'item', 'Element Code':'element_code', 'Element':'element', 'Year':'year', 'Unit':'unit', 'Value':'value', 'Flag':'flag'}, inplace=True)

In [159]:
food_balances['area'].value_counts()

area
World                                      26494
Asia                                       25806
Americas                                   25407
Net Food Importing Developing Countries    25351
Africa                                     25024
                                           ...  
South Sudan                                 4591
Bhutan                                      4556
Micronesia (Federated States of)            3575
Nauru                                       3470
Netherlands Antilles (former)                141
Name: count, Length: 221, dtype: int64

In [160]:
# Check the date range of the table
food_balances['year'].value_counts()

year
2021    369154
2020    368661
2019    366960
2017    359076
2018    358735
2016    357987
2014    357970
2015    357899
2013    357396
2012    357061
2010    355189
2011    354820
Name: count, dtype: int64

Since this data only covers the years between 2010-2021, we want to combine it with an additional dataset covering the years before. 

In [161]:
url = f'https://fenixservices.fao.org/faostat/static/bulkdownloads/FoodBalanceSheetsHistoric_E_All_Data_(Normalized).zip'
zip_file = 'FoodBalanceSheetsHistoric_E_All_Data_(Normalized).zip'
csv_file = 'FoodBalanceSheetsHistoric_E_All_Data_(Normalized).csv'

In [65]:
# Run the data download function 
data_download()

In [162]:
# Import crops emissions data for all countries
food_balances_hist = pd.read_csv(path+csv_file, encoding='latin-1', low_memory=False)

In [163]:
food_balances_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11479903 entries, 0 to 11479902
Data columns (total 13 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Area Code        int64  
 1   Area Code (M49)  object 
 2   Area             object 
 3   Item Code        int64  
 4   Item Code (FBS)  object 
 5   Item             object 
 6   Element Code     int64  
 7   Element          object 
 8   Year Code        int64  
 9   Year             int64  
 10  Unit             object 
 11  Value            float64
 12  Flag             object 
dtypes: float64(1), int64(5), object(7)
memory usage: 1.1+ GB


In [78]:
# Since the info above does not provide anything apart from the data type, get additional info on unique and missing values
print("Unique values and missing values(%) of each column:")

food_balances_hist_info= pd.DataFrame({"Unique values": food_balances_hist.nunique(),
                            "Missing values(%)": round(food_balances_hist.isnull().sum()/food_balances_hist.shape[0]*100, 2)
                            }).rename_axis('Columns', axis='rows')                       

food_balances_hist_info

Unique values and missing values(%) of each column:


,Unique values,Missing values(%)
Columns,,
Area Code,217,0.0
Area Code (M49),217,0.0
Area,217,0.0
Item Code,123,0.0
Item Code (FBS),123,0.0
Item,120,0.0
Element Code,16,0.0
Element,16,0.0
Year Code,53,0.0


In [81]:
# Check for full duplicates
food_balances_hist.duplicated().value_counts()

False    11479903
Name: count, dtype: int64

In [167]:
food_balances_hist.head()

,area_code,area,item_code,item,element_code,element,year,unit,value,flag
0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1961,1000 No,8954.0,X
1,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1962,1000 No,9142.0,X
2,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1963,1000 No,9340.0,X
3,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1964,1000 No,9547.0,X
4,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1965,1000 No,9765.0,X


In [165]:
# Drop unneeded columns
food_balances_hist.drop(columns = ['Year Code', 'Area Code (M49)', 'Item Code (FBS)'], inplace=True)

In [166]:
# Rename remaining columns
food_balances_hist.rename(columns={'Area Code':'area_code', 'Area':'area', 'Item Code':'item_code', 'Item':'item', 'Element Code':'element_code', 'Element':'element', 'Year':'year', 'Unit':'unit', 'Value':'value', 'Flag':'flag'}, inplace=True)

Looking at the number of unique values in food_balances and food_balances_hist, we see that the number of unique values in Area and Element columns differs  - to be taken into account when merging the dataframes.

In [168]:
food_balances_combined = pd.concat([food_balances,food_balances_hist], ignore_index=True)

In [169]:
food_balances_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15800811 entries, 0 to 15800810
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   area_code     int64  
 1   area          object 
 2   item_code     int64  
 3   item          object 
 4   element_code  int64  
 5   element       object 
 6   year          int64  
 7   unit          object 
 8   value         float64
 9   flag          object 
dtypes: float64(1), int64(4), object(5)
memory usage: 1.2+ GB


In [170]:
food_balances_combined.head()

,area_code,area,item_code,item,element_code,element,year,unit,value,flag
0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,2010,1000 No,28189.67,X
1,2,Afghanistan,2501,Population,511,Total Population - Both sexes,2011,1000 No,29249.16,X
2,3,Albania,2501,Population,511,Total Population - Both sexes,2010,1000 No,2913.40,X
3,3,Albania,2501,Population,511,Total Population - Both sexes,2011,1000 No,2900.65,X
4,2,Afghanistan,2501,Population,511,Total Population - Both sexes,2012,1000 No,30466.48,X


In [171]:
# Check for duplicates after combining the tables
food_balances_combined.duplicated().value_counts()

False    15491348
True       309463
Name: count, dtype: int64

In [178]:
# Dropping duplicated rows
food_balances_combined.drop_duplicates(inplace=True)

In [199]:
# engine = get_engine()

NameError: name 'dotenv_values' is not defined

In [181]:
'''''
# Export the table
engine = get_engine()
schema = 'capstone_envirolytics'
table_name = 'food_balances'

if engine!=None:
    try:
        food_balances_combined.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schema that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
'''''

NameError: name 'get_engine' is not defined